In [7]:
import os
import pickle
import pandas as pd
from pathlib import Path

In [9]:
# 피클 파일
path = "/Users/pjy97/Desktop/공부/NLP스터디/data/pickle_180_days_compare_kospi/yes_data/"
data_list = os.listdir(path)
len(data_list)

data_dict = {}


for data_name in data_list:
    with open(path + data_name, 'rb') as f:
        data_dict[data_name] = pickle.load(f)

In [16]:
imp = data_dict['20171215.pkl'].loc[:, 0:5]

In [46]:
imp = data_dict['20171215.pkl'].loc[:, 0:].astype(float)

In [51]:
imp.transpose().corr()

주가번호,393929,394109,393906,393966,394151,393918,394044,394083,395295,393952,...,393956,394006,393857,393928,394142,394051,393864,393984,393872,393933
주가번호,,,,,,,,,,,,,,,,,,,,,
393929,1.000000,0.319159,-0.585353,-0.585353,-0.044792,0.331748,-0.055021,0.466685,-0.024558,0.360574,...,0.483632,0.483632,0.483632,0.588047,0.330686,0.090452,0.482539,0.254147,0.838181,0.722506
394109,0.319159,1.000000,-0.268156,-0.268156,-0.202244,0.317355,-0.196748,0.645672,-0.108132,0.439815,...,0.234799,0.234799,0.234799,0.424877,0.147458,-0.014751,0.101390,0.579787,0.246301,0.391575
393906,-0.585353,-0.268156,1.000000,1.000000,0.311092,-0.444623,0.365917,-0.536000,-0.021949,-0.377990,...,-0.089825,-0.089825,-0.089825,-0.561710,-0.463273,0.143058,-0.671819,-0.205584,-0.691463,-0.591976
393966,-0.585353,-0.268156,1.000000,1.000000,0.311092,-0.444623,0.365917,-0.536000,-0.021949,-0.377990,...,-0.089825,-0.089825,-0.089825,-0.561710,-0.463273,0.143058,-0.671819,-0.205584,-0.691463,-0.591976
394151,-0.044792,-0.202244,0.311092,0.311092,1.000000,-0.485418,0.508721,-0.089207,0.389707,-0.384574,...,0.274689,0.274689,0.274689,-0.175879,-0.314380,0.222281,0.220668,-0.077162,-0.097578,-0.114680
393918,0.331748,0.317355,-0.444623,-0.444623,-0.485418,1.000000,-0.618059,0.545203,-0.405290,0.760998,...,-0.133609,-0.133609,-0.133609,0.446265,0.384230,-0.407048,0.327340,0.152359,0.611099,0.418768
394044,-0.055021,-0.196748,0.365917,0.365917,0.508721,-0.618059,1.000000,-0.306565,0.567219,-0.521384,...,0.261846,0.261846,0.261846,0.048414,0.034593,0.655971,-0.129977,0.176406,-0.332521,0.128502
394083,0.466685,0.645672,-0.536000,-0.536000,-0.089207,0.545203,-0.306565,1.000000,0.073308,0.600335,...,0.009360,0.009360,0.009360,0.431064,0.306700,-0.233257,0.500564,0.314535,0.522378,0.459307
395295,-0.024558,-0.108132,-0.021949,-0.021949,0.389707,-0.405290,0.567219,0.073308,1.000000,-0.153828,...,-0.101377,-0.101377,-0.101377,0.042753,0.362056,0.341166,0.354872,0.003921,-0.186387,0.086458


In [22]:
data = {
    'Feature1': [1, 5, 2, 4, 7],
    'Feature2': [5, 4, 3, 2, 9],
    'Feature3': [2, 3, 4, 1, 8]
}
df = pd.DataFrame(data, index=['Row1', 'Row2', 'Row3', 'Row4', 'Row5'])

In [24]:
dfcorrelation_matrix_rows = df.transpose().corr()

In [25]:
dfcorrelation_matrix_rows

,Row1,Row2,Row3,Row4,Row5
Row1,1.000000,-0.240192,0.240192,-0.419314,0.960769
Row2,-0.240192,1.000000,-1.000000,0.981981,-0.500000
Row3,0.240192,-1.000000,1.000000,-0.981981,0.500000
Row4,-0.419314,0.981981,-0.981981,1.000000,-0.654654
Row5,0.960769,-0.500000,0.500000,-0.654654,1.000000
